In [100]:
from utils import *
import numpy as np
from skimage import measure
import os
import cv2

In [101]:
def project_vertices(vertices, frame_matrix, image_size):
    # Берём только 3х3 матрицу вращения и 3х1 вектор смещения
    R = frame_matrix[:3, :3]
    T = frame_matrix[:3, 3]
    f = 1.0
    H = np.vstack([R * f, T])

    # Ортографическая проекция
    projected = vertices @ H[:3, :3].T + H[:3, 2]
    projected = projected[:, :2]

    # Масштабируем до размера изображения
    projected -= projected.min(axis=0)
    projected /= projected.max(axis=0)
    projected *= np.array(image_size)[::-1]  # W x H
    return projected


def build_triangle_index_map(mesh, image_size):
    vertices = mesh["vertices"]
    faces = mesh["faces"]
    frame_matrix = mesh["frame_matrix"]
    projected_vertices = project_vertices(vertices, frame_matrix, image_size)
    index_map = np.full(image_size[::-1], -1, dtype=np.int32)  # (H, W)

    for tri_idx, face in enumerate(faces):
        pts = projected_vertices[face].astype(np.int32)
        cv2.fillPoly(index_map, [pts], color=tri_idx)

    return index_map

1. Считываем данные и приводим к единой системе координат

In [174]:
# Размер воксельного пространства
VOXEL_GRID_SIZE = 256
# Размер проекции
image_size = (VOXEL_GRID_SIZE * 2, VOXEL_GRID_SIZE * 2)

In [175]:
# Получим пути к исходным файлам
files = [f for f in os.listdir("data") if f.endswith('.x')]

In [176]:
meshes = []
for f in files:
    print(f)
    filepath = os.path.abspath(f'data\\{f}')
    data = parse_x_file(filepath)

    # Переведём координаты точек из локальной системы координат в систему координат камеры
    data['vertices'] = apply_transformation(data.get('vertices'), data.get('frame_matrix'))
    
    # Предварительная индексация треугольников
    data["index_map"] = build_triangle_index_map(data, image_size)
    
    # Путь к текстуре
    data['texture_path'] = os.path.abspath(f'data\\{f.rstrip(".x")}.bmp')
    meshes.append(data)

teapot_1.x
num_vertices: 200310
num_faces: 379304
num_uvs: 200310
teapot_2.x
num_vertices: 190307
num_faces: 358169
num_uvs: 190307


In [90]:
# Конфертируем все меши в PointCloud
pcds_raw = [vertices_to_pcd(m.get('vertices')) for m in meshes]
# Считаем нормали для PointCloud
pcds_down = [get_normals_to_pcd(pcd) for pcd in pcds_raw]

# Посмотрим как взаиморасположены меши
o3d.visualization.draw_geometries(pcds_down)

2. Инициализируем воксельное пространство

In [158]:
# Сбор всех вершин в единую структуру
all_vertices = np.vstack([mesh["vertices"] for mesh in meshes])

In [159]:
# Границы сцены
min_bound = np.min(all_vertices, axis=0)
max_bound = np.max(all_vertices, axis=0)
scene_size = max_bound - min_bound

In [160]:
# Размер одного вокселя
voxel_size = scene_size / VOXEL_GRID_SIZE

In [161]:
# Инициализация скалярного поля и весов
D = np.zeros((VOXEL_GRID_SIZE + 1, VOXEL_GRID_SIZE + 1, VOXEL_GRID_SIZE + 1), dtype=np.float32)
W = np.zeros_like(D)

In [162]:
# Сохраняем параметры для дальнейших преобразований
voxel_space = {
    "min_bound": min_bound,
    "max_bound": max_bound,
    "scene_size": scene_size,
    "voxel_size": voxel_size,
    "D": D,
    "W": W,
    "grid_size": VOXEL_GRID_SIZE
}

3. Объединяем меши воксельным методом

In [165]:
def compute_triangle_normal(p0, p1, p2):
    return np.cross(p1 - p0, p2 - p0)


def point_to_triangle_distance(p, a, b, c):
    # Истинное расстояние от точки до треугольника в 3D
    ab = b - a
    ac = c - a
    ap = p - a
    
    d1 = np.dot(ab, ap)
    d2 = np.dot(ac, ap)
    if d1 <= 0.0 and d2 <= 0.0:
        # точка p лежит вне треугольника со стороны вершины a
        return np.linalg.norm(ap)

    bp = p - b
    d3 = np.dot(ab, bp)
    d4 = np.dot(ac, bp)
    if d3 >= 0.0 and d4 <= d3:
        # точка p лежит вне треугольника со стороны вершины b
        return np.linalg.norm(bp)

    vc = d1*d4 - d3*d2
    if vc <= 0.0 and d1 >= 0.0 and d3 <= 0.0:
        v = d1 / (d1 - d3)
        proj = a + v * ab
        # точка p ближе всего к ребру AB.
        return np.linalg.norm(p - proj)

    cp = p - c
    d5 = np.dot(ab, cp)
    d6 = np.dot(ac, cp)
    if d6 >= 0.0 and d5 <= d6:
        # точка p лежит вне треугольника со стороны вершины c
        return np.linalg.norm(cp)

    vb = d5*d2 - d1*d6
    if vb <= 0.0 and d2 >= 0.0 and d6 <= 0.0:
        w = d2 / (d2 - d6)
        proj = a + w * ac
        # точка p ближе всего к ребру AC
        return np.linalg.norm(p - proj)

    va = d3*d6 - d5*d4
    if va <= 0.0 and (d4 - d3) >= 0.0 and (d5 - d6) >= 0.0:
        w = (d4 - d3) / ((d4 - d3) + (d5 - d6))
        proj = b + w * (c - b)
        # точка p ближе всего к ребру BC.
        return np.linalg.norm(p - proj)

    # точка p лежит внутри треугольника
    n = np.cross(ab, ac)
    n /= np.linalg.norm(n)
    return abs(np.dot(ap, n))


def compute_weight(p, camera_center, normal):
    direction = camera_center - p
    direction /= np.linalg.norm(direction)
    weight = abs(np.dot(direction, normal))
    return weight


def integrate_mesh_to_voxel_grid(mesh, voxel_space):
    vertices = mesh["vertices"]
    faces = mesh["faces"]
    frame_matrix = mesh["frame_matrix"]
    camera_center = apply_transformation(np.array([[0.0, 0.0, 0.0]]), frame_matrix)[0]

    D = voxel_space["D"]
    W = voxel_space["W"]
    min_bound = voxel_space["min_bound"]
    voxel_size = voxel_space["voxel_size"]

    for face in tqdm(faces, desc="Faces", unit=" triangles", unit_scale=1):
        a, b, c = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        normal = compute_triangle_normal(a, b, c)
        normal /= np.linalg.norm(normal)

        # Bounding box треугольника в МИРОВЫХ координата
        tri_min = np.minimum(np.minimum(a, b), c)
        tri_max = np.maximum(np.maximum(a, b), c)
        
        # Переводим его в индексы воксельной сетки
        min_idx = np.floor((tri_min - min_bound) / voxel_size).astype(int)
        max_idx = np.ceil((tri_max - min_bound) / voxel_size).astype(int)

        for i in range(min_idx[0], max_idx[0] + 1):
            for j in range(min_idx[1], max_idx[1] + 1):
                for k in range(min_idx[2], max_idx[2] + 1):
                    voxel_center = min_bound + voxel_size * (np.array([i, j, k]) + 0.5)
                    distance = point_to_triangle_distance(voxel_center, a, b, c)
                    weight = compute_weight(voxel_center, camera_center, normal)

                    if weight == 0.0:
                        continue  # скипаем неинтересные места

                    D[i, j, k] = (W[i, j, k] * D[i, j, k] + weight * distance) / (W[i, j, k] + weight)
                    W[i, j, k] += weight


In [166]:
# Вычислим интегральную функцию растояния и весовую функцию для каждого вокселя
for mesh in meshes:
    integrate_mesh_to_voxel_grid(mesh, voxel_space)

Faces: 100%|██████████| 379k/379k [03:04<00:00, 2.05k triangles/s] 
Faces: 100%|██████████| 358k/358k [02:41<00:00, 2.22k triangles/s] 


In [167]:
safe_D = np.copy(D)

In [169]:
# Marching cubes ищет поверхность, где D == 0
verts, faces, normals, values = measure.marching_cubes(safe_D, level=0, spacing=(voxel_space["voxel_size"]))

In [170]:
# Создадим меш на основе вершин и треугольников
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

In [171]:
# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

TriangleMesh with 275434 points and 548494 triangles.

In [173]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([mesh])

4. Сохраняем полученный меш

In [156]:
write_mesh_to_x(mesh, 'output_mesh.x', save_normals=False, save_texture=False)

Vertices: 100%|██████████| 181k/181k [00:01<00:00, 149k points/s] 
Faces: 100%|██████████| 379k/379k [00:01<00:00, 324k triangles/s] 
